In [ ]:
# Install required libraries
!pip install transformers torch scikit-learn tqdm wget

In [ ]:

!pip uninstall tensorflow tensorflow-gpu tf-keras -y
!pip install tensorflow==2.19.0

# Verify the installation
import tensorflow as tf
print(f"TensorFlow version: {tf.__version__}")
print("✅ TensorFlow 2.19.0 installed successfully!")

# Also install tensorflow-text for compatibility
!pip install tensorflow-text==2.19.0

Found existing installation: tensorflow 2.19.0
Uninstalling tensorflow-2.19.0:
  Successfully uninstalled tensorflow-2.19.0
  Using cached tensorflow-2.19.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
Using cached tensorflow-2.19.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (645.0 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.12.0 requires tf_keras~=2.19, which is not installed.
dopamine-rl 4.1.2 requires tf-keras>=2.18.0, which is not installed.
TensorFlow version: 2.19.0
✅ TensorFlow 2.19.0 installed successfully!


### Load Data

In [ ]:
!git clone https://github.com/nlpdata/dialogre.git
%cd dialogre

fatal: destination path 'dialogre' already exists and is not an empty directory.
/content/dialogre


In [ ]:
!ls data_v2/en/data

dev.json  test.json  train.json


In [ ]:
!mkdir -p data
!cp data_v2/en/data/* data/
!ls data

dev.json  test.json  train.json


## Downloading the pretrained model

In [ ]:
from transformers import BertModel, BertTokenizer

# Path where we’ll keep the model files
BERT_BASE_DIR = "/content/dialogre/bert/uncased_L-12_H-768_A-12"

# Download pretrained BERT (PyTorch version)
model = BertModel.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Save them to the folder
model.save_pretrained(BERT_BASE_DIR)
tokenizer.save_pretrained(BERT_BASE_DIR)

# Verify
!ls -la $BERT_BASE_DIR

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


total 1286692
drwxr-xr-x 2 root root      4096 Aug 29 12:20 .
drwxr-xr-x 6 root root      4096 Aug 29 11:49 ..
-rw-r--r-- 1 root root       313 Aug 29 11:00 bert_config.json
-rw-r--r-- 1 root root 440425712 Aug 29 11:00 bert_model.ckpt.data-00000-of-00001
-rw-r--r-- 1 root root      8528 Aug 29 11:00 bert_model.ckpt.index
-rw-r--r-- 1 root root    904243 Aug 29 11:00 bert_model.ckpt.meta
-rw-r--r-- 1 root root       618 Aug 29 12:43 config.json
-rw-r--r-- 1 root root 437951328 Aug 29 12:43 model.safetensors
-rw-r--r-- 1 root root 438005695 Aug 29 12:20 pytorch_model.bin
-rw-r--r-- 1 root root       125 Aug 29 12:43 special_tokens_map.json
-rw-r--r-- 1 root root      1272 Aug 29 12:43 tokenizer_config.json
-rw-r--r-- 1 root root    231508 Aug 29 12:43 vocab.txt


## Prepare Data Directory in bert folder

In [ ]:
bert_data_dir = "/content/dialogre/bert/data"
if not os.path.exists(bert_data_dir):
    print("Copying data directory to bert/ folder...")
    !cp -r /content/dialogre/data_v2/en/data /content/dialogre/bert/
    print("Data copied successfully!")
else:
    print("Data directory already exists in bert/ folder")

print("Contents of bert/data directory:")
!ls -la /content/dialogre/bert/data/

Data directory already exists in bert/ folder
Contents of bert/data directory:
total 3740
drwxr-xr-x 2 root root    4096 Aug 29 11:00 .
drwxr-xr-x 6 root root    4096 Aug 29 11:49 ..
-rw-r--r-- 1 root root  751932 Aug 29 11:00 dev.json
-rw-r--r-- 1 root root  725696 Aug 29 11:00 test.json
-rw-r--r-- 1 root root 2338606 Aug 29 11:00 train.json


## Convert TF Checkpoint to PyTorch Format


In [ ]:
conversion_script_path = "/content/dialogre/bert/convert_tf_checkpoint_to_pytorch.py"

# Define paths for the conversion
tf_checkpoint_path = os.path.join(BERT_BASE_DIR, "bert_model.ckpt")
bert_config_file = os.path.join(BERT_BASE_DIR, "bert_config.json")
pytorch_dump_path = os.path.join(BERT_BASE_DIR, "pytorch_model.bin")

print(f"TF checkpoint path: {tf_checkpoint_path}")
print(f"BERT config file: {bert_config_file}")
print(f"PyTorch output path: {pytorch_dump_path}")

# Run the conversion script exactly as specified in the README
print("Running conversion script...")
!cd /content/dialogre/bert/ && python convert_tf_checkpoint_to_pytorch.py \
  --tf_checkpoint_path="$BERT_BASE_DIR/bert_model.ckpt" \
  --bert_config_file="$BERT_BASE_DIR/bert_config.json" \
  --pytorch_dump_path="$BERT_BASE_DIR/pytorch_model.bin"

print("Conversion completed!")
print("Files in BERT directory after conversion:")
!ls -la "{BERT_BASE_DIR}"

TF checkpoint path: /content/dialogre/bert/uncased_L-12_H-768_A-12/bert_model.ckpt
BERT config file: /content/dialogre/bert/uncased_L-12_H-768_A-12/bert_config.json
PyTorch output path: /content/dialogre/bert/uncased_L-12_H-768_A-12/pytorch_model.bin
Running conversion script...
2025-08-29 12:43:55.688728: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1756471435.720555   27148 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1756471435.730098   27148 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1756471435.752572   27148 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same targe

## Training Session

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"PyTorch will use device: {device}")

PyTorch will use device: cuda


In [ ]:
import os
if 'BERT_BASE_DIR' not in os.environ:
    os.environ['BERT_BASE_DIR'] = '/content/dialogre/bert/uncased_L-12_H-768_A-12'
    print("Setting BERT_BASE_DIR environment variable:", os.environ['BERT_BASE_DIR'])
else:
    print("BERT_BASE_DIR environment variable is already set:", os.environ['BERT_BASE_DIR'])

# Execute the training, evaluation, and evaluation script
!python ./bert/run_classifier.py \
  --task_name bert \
  --do_train --do_eval \
  --data_dir . \
  --vocab_file $BERT_BASE_DIR/vocab.txt \
  --bert_config_file $BERT_BASE_DIR/bert_config.json \
  --init_checkpoint $BERT_BASE_DIR/pytorch_model.bin \
  --max_seq_length 512 \
  --train_batch_size 24 \
  --learning_rate 3e-5 \
  --num_train_epochs 20.0 \
  --output_dir bert_f1 \
  --gradient_accumulation_steps 2 \
&& rm bert_f1/model_best.pt \
&& cp -r bert_f1 bert_f1c \
&& python ./bert/run_classifier.py \
  --task_name bertf1c \
  --do_eval \
  --data_dir . \
  --vocab_file $BERT_BASE_DIR/vocab.txt \
  --bert_config_file $BERT_BASE_DIR/bert_config.json \
  --init_checkpoint $BERT_BASE_DIR/pytorch_model.bin \
  --max_seq_length 512 \
  --train_batch_size 24 \
  --learning_rate 3e-5 \
  --num_train_epochs 20.0 \
  --output_dir bert_f1c \
  --gradient_accumulation_steps 2 \
&& python evaluate.py \
  --f1dev bert_f1/logits_dev.txt \
  --f1test bert_f1/logits_test.txt \
  --f1cdev bert_f1c/logits_dev.txt \
  --f1ctest bert_f1c/logits_test.txt

BERT_BASE_DIR environment variable is already set: /content/dialogre/bert/uncased_L-12_H-768_A-12
08/29/2025 12:44:16 - INFO - __main__ -   device cuda n_gpu 1 distributed training False
08/29/2025 12:44:16 - INFO - __main__ -   5997,1914,1862
#examples 1914
08/29/2025 12:44:19 - INFO - __main__ -   *** Example ***
08/29/2025 12:44:19 - INFO - __main__ -   guid: dev-0
08/29/2025 12:44:19 - INFO - __main__ -   tokens: [CLS] speaker 1 : hey ! speaker 2 : hey . speaker 3 : hey , man . what ' s up ? speaker 1 : maybe you can tell me . my agent would like to know why i didn ' t show up at the audition i didn ' t know i had today . the first good thing she gets me in weeks . how could you not give me the message ? ! speaker 3 : well , i ' ll tell ya i do enjoy guilt , but , ah , it wasn ' t me . speaker 2 : yes , it was ! it was him ! uh huh ! okay , it was me ! speaker 1 : how is it you ? speaker 2 : well , it was just , it was all so crazy , you know . i mean , chandler was in the closet ,